In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_general6_FL_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['general_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

#### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate = learning_rate,
                       depth = depth,
                       estimators=estimators,
                       holdout_split=holdout_split,
                       seed=816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binh

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.56105930728919, 0.08289599608368595)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.6215273766036412, 0.052869124854952765)

## Lasso Stumps

In [111]:
## load whole data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])
X_stumps, Y_stumps = data.loc[:,:'five_year1'], data['general_six_month'].values
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_test_stumps.csv")
X_train_stumps, Y_train_stumps = train_stumps.loc[:,:'five_year1'], train_stumps['general_six_month'].values
X_test_stumps, Y_test_stumps = test_stumps.loc[:,:'five_year1'], test_stumps['general_six_month'].values
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [46]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.08, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

17

In [47]:
stumps.latex_stump_table(single_stump_model['coefs'], 
                         single_stump_model['features'], 
                         single_stump_model['intercept'], 
                         single_stump_model['dictionary'])

\begin{{tabular}}{{|l|r|r|}} \hline
1. age_at_current_charge19 & 0.01864444266312891 & -... \\ \hline
2. age_at_current_charge23 & 0.23743232726591834 & -... \\ \hline
3. age_at_current_charge30 & 0.12519473804982992 & -... \\ \hline
4. age_at_current_charge37 & 0.22652871985062956 & -... \\ \hline
5. age_at_first_charge29 & 0.1694487219116978 & -... \\ \hline
6. age_at_first_charge33 & 0.0004422526065423885 & -... \\ \hline
7. p_incarceration1 & 0.1253632299544665 & +... \\ \hline
8. p_weapon1 & 0.0026446418290414758 & +... \\ \hline
9. p_misdemeanor3 & 0.01129964802860267 & +... \\ \hline
10. p_misdemeanor5 & 0.2924521937965682 & +... \\ \hline
11. p_misdemeanor6 & 0.03156239392090238 & +... \\ \hline
12. p_violence2 & 0.09020031517519694 & +... \\ \hline
13. p_property2 & 0.04952669151760761 & +... \\ \hline
14. p_dui1 & 0.2588009421573325 & -... \\ \hline
15. six_month1 & 0.191678496135951 & +... \\ \hline
16. one_year1 & 0.08702021822776024 & +... \\ \hline
17. three_year1 & 0.109

#### Nested Cross Validation

In [11]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [12]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.08}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}],
 0.6192074866665782,
 0.04306450520667093)

## RiskSLIM

In [112]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]

## train & test data
train_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_train_stumps.csv")
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_test_stumps.csv")

In [113]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.05,
                                      columns=cols, 
                                      seed=816)
len(best_stump_model['features'])

14

In [114]:
selected_features = best_stump_model['features']

In [115]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['general_six_month', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['general_six_month', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0
    
sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [116]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='general_six_month', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000, 
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/11/19 @ 04:49 PM | 193 rows in lookup table
11/11/19 @ 04:49 PM | ------------------------------------------------------------
11/11/19 @ 04:49 PM | runnning initialization procedure
11/11/19 @ 04:49 PM | ------------------------------------------------------------
11/11/19 @ 04:49 PM | CPA produced 2 cuts
11/11/19 @ 04:49 PM | running naive rounding on 4 solutions
11/11/19 @ 04:49 PM | best objective value: 0.6931
11/11/19 @ 04:49 PM | rounding produced 4 integer solutions
11/11/19 @ 04:49 PM | best objective value is 0.6931
11/11/19 @ 04:49 PM | running sequential rounding on 4 solutions
11/11/19 @ 04:49 PM | best objective value: 0.6931
11/11/19 @ 04:49 PM | sequential rounding produced 0 integer solutions
11/11/19 @ 04:49 PM | polishing 4 solutions
11/11/19 @ 04:49 PM | best objective value: 0.6931
11/11/19 @ 04:49 PM | polishing produced 4 integer solutions
11/11/19 @ 04:49 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:49 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5291.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:49 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5381.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:49 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5301.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:49 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5203.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:53 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5267.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:53 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5106.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:53 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5133.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 04:53 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5216.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [117]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.5822775723438409, 0.5839618976131931)

### Single RiskSLIM Model

In [34]:
selected_features = ["general_six_month"] + best_stump_model['features']
sub_train_data = train_data[selected_features]
sub_test_data = test_data[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [35]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': cols,
    'outcome_name': 'general_six_month',
    'sample_weights': sample_weights
}

In [36]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/11/19 @ 03:46 PM | 223 rows in lookup table
11/11/19 @ 03:46 PM | ------------------------------------------------------------
11/11/19 @ 03:46 PM | runnning initialization procedure
11/11/19 @ 03:46 PM | ------------------------------------------------------------
11/11/19 @ 03:46 PM | CPA produced 2 cuts
11/11/19 @ 03:46 PM | running naive rounding on 4 solutions
11/11/19 @ 03:46 PM | best objective value: 0.6931
11/11/19 @ 03:46 PM | rounding produced 4 integer solutions
11/11/19 @ 03:46 PM | best objective value is 0.6931
11/11/19 @ 03:46 PM | running sequential rounding on 4 solutions
11/11/19 @ 03:46 PM | best objective value: 0.6931
11/11/19 @ 03:46 PM | sequential rounding produced 0 integer solutions
11/11/19 @ 03:46 PM | polishing 4 solutions
11/11/19 @ 03:46 PM | best objective value: 0.6931
11/11/19 @ 03:46 PM | polishing produced 4 integer solutions
11/11/19 @ 03:46 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/11/19 @ 03:46 PM | adding 261 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5156.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

In [37]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [38]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.5934784698193003

In [39]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.53450337512054

## Arnold PSA

In [40]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['general_six_month'].values

In [41]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    try:
        arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['arnold_nca'],
                                                   labels = test_y)
        arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
        race_auc.append(arnold_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = []
try:
    arnold_race_auc = pd.concat(race_auc, ignore_index=True)
    arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    arnold_race_auc = arnold_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

## Compass

In [43]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Recidivism_decile_score', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['general_six_month'].values

In [44]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
compas_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    compas_auc.append(roc_auc_score(test_y, test_x['Risk of Recidivism_decile_score'].values))
    
    ################################
    ## race_auc
    try:
        compas_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['Risk of Recidivism_decile_score'],
                                                   labels = test_y)
        compas_race_auc_final = compas_race_auc.assign(fold_num = [i]*compas_race_auc['Attribute'].count())
        race_auc.append(compas_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    compas_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['Risk of Recidivism_decile_score'],
                                                                         labels = test_y)
    compas_condition_pn_final = compas_condition_pn.assign(fold_num = [i]*compas_condition_pn['Attribute'].count())
    condition_pn.append(compas_condition_pn_final)
    
    ################################
    ## no condition pn
    compas_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['Risk of Recidivism_decile_score'],
                                                                labels = test_y)
    compas_no_condition_pn_final = compas_no_condition_pn.assign(fold_num = [i]*compas_no_condition_pn['Attribute'].count())
    no_condition_pn.append(compas_no_condition_pn_final)
    
    i += 1
    
## race_auc
compas_race_auc = []
try:  
    compas_race_auc = pd.concat(race_auc, ignore_index=True)
    compas_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    compas_race_auc = compas_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
compas_condition_pn = pd.concat(condition_pn, ignore_index=True)
compas_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_condition_pn = compas_condition_pn.reset_index(drop=True)

## no_condition_pn
compas_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
compas_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_no_condition_pn = compas_no_condition_pn.reset_index(drop=True)

### Results

In [164]:
#### save results
summary_general6_FL_interpret = {"CART": cart_summary,
                                 "EBM": ebm_summary, 
                                 'Lasso Stumps': stump_summary, 
                                 'RiskSLIM': riskslim_summary, 
                                 'Arnold PSA': arnold_auc, 
                                 'Compas': compas_auc}
%store summary_general6_FL_interpret

Stored 'summary_general6_FL_interpret' (dict)


In [103]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)], 
           ['Compas', round(np.mean(compas_auc), 3)]]
results

[['CART', 0.56105930728919, 0.08289599608368595],
 ['EBM', 0.6215273766036412, 0.052869124854952765],
 ['Lasso Stumps', 0.6192074866665782, 0.04306450520667093],
 ['RiskSLIM', 0.5839618976131931],
 ['Arnold PSA', 0.577],
 ['Compas', 0.609]]

In [104]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [118]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/model results/Interpretable Models/Six Month/"
results = [["", "CART", "EBM", "Lasso Stumps", "RiskSLIM", "Performance Range", "Arnold PSA", "Compas"],
           ["General", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")", 
            np.str(round(np.mean(compas_auc), 3)) + " (" + np.str(round(np.std(compas_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Confusion Matrix

In [106]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/general/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/general/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/general/riskslim_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/general/stumps_confusion.csv', index=None,header=True)

### Calibration

In [107]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

## save results
cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/general/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/general/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/general/riskslim_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/general/stumps_calibration.csv', index=None,header=True)

### Race AUC

In [108]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

## save results
cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/riskslim_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/arnold_race_auc.csv', index=None,header=True)
compas_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/general/compas_race_auc.csv', index=None,header=True)

### Condition PN

In [109]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

## save results
cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/riskslim_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/arnold_condition_pn.csv', index=None,header=True)
compas_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/general/compas_condition_pn.csv', index=None,header=True)

### No Condition PN

In [110]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

## save results
cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/riskslim_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/arnold_no_condition_pn.csv', index=None,header=True)
compas_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/general/compas_no_condition_pn.csv', index=None,header=True)